<a href="https://colab.research.google.com/github/EthanBores/OptAI/blob/main/option_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make imports

In [ ]:
# Required Imports
!pip install yahoo_fin
!pip install neptune-client==0.9.8

import neptune
import os
import nltk
nltk.download('vader_lexicon')
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import scipy.stats as st

from datetime import timedelta
from datetime import date
from bs4 import BeautifulSoup
from yahoo_fin import options
from yahoo_fin import news
from yahoo_fin import stock_info
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 5.3 MB/s 
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |████████████████████████████████| 112 kB 45.2 MB/s 
     |████████████████████████████████| 127 kB 14.0 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=59e00623ed8cc9307dd6984689cee1691e3dabedb13a3d6a8a1e2fd3922250d2
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=2bedac750a49e4c8af0675716eb7c093df8d26ddf9218473dbf44d57e2519a00
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=4b60b73c764f72f5f3e872b3a651fb88fa955502e53c67a38fa2b

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
# Get user input of ticker symbol

ticker = input("Enter stock ticker symbol: ")
sym_name = stock_info.get_quote_data(ticker)['shortName']
print("Retrieving data for symbol ",ticker.upper(),"/ "+sym_name+"...")

##################################
##### General Data Retrieval #####
##################################

# Retrieve mass of data for options current quote

chain = options.get_options_chain(ticker)
q_data = stock_info.get_quote_data(ticker)
live_price = stock_info.get_live_price(ticker)
vix = stock_info.get_quote_data('^VIX')['regularMarketPrice']

# Collect most important quote data

try:
  analyst_Rating = q_data["averageAnalystRating"]
  AR_num = float(analyst_Rating[0:3])


except KeyError:
  print("Analyst rating unavailable... Defualting to 3")
  AR_num = 3

# Collect and organize relevant columns from data options chain table

calls = chain['calls']
puts = chain['puts']

calls_LP     = calls.loc[:, "Last Price"]
puts_LP      = puts.loc[:, "Last Price"]

calls_Strike = calls.loc[:, "Strike"]
puts_Strike  = puts.loc[:, "Strike"]

calls_IV     = calls.loc[:, "Implied Volatility"]
puts_IV      = puts.loc[:, "Implied Volatility"]

calls_Name   = calls.loc[:, "Contract Name"]
puts_Name    = puts.loc[:, "Contract Name"]

# Pull expiration dates from 'Contract Name' column and store to list

calls_Exp    = []
puts_Exp     = []

calls_Name.to_list

for i in calls_Name:
  calls_Exp.append(re.findall(r'(\d{6})', i))
for i in puts_Name:
  puts_Exp.append(re.findall(r'(\d{6})', i))

print('\n')
print("Market Price:            $",q_data['regularMarketPrice'])
print("Percent Change:           ",q_data['regularMarketChangePercent'])
print("Regular Market Day High: $",q_data['regularMarketDayHigh'])
print("Regular Market Day Low:  $",q_data['regularMarketDayLow'])
print("Volume:                   ",q_data['regularMarketVolume'],'\n')

print("Market Volatility Index:  ", vix)

Enter stock ticker symbol: aapl
Retrieving data for symbol  AAPL / Apple Inc....


Market Price:            $ 131.56
Percent Change:            1.1533139
Regular Market Day High: $ 133.079
Regular Market Day Low:  $ 129.81
Volume:                    134520290 

Market Volatility Index:   31.13


In [ ]:
###########################
##### Data Repackager #####
###########################

calls = {'Contract Name':[], 'Expiration':[], 'Last Price':[], 'Strike':[], 'IV':[]}
puts = {'Contract Name':[], 'Expiration':[], 'Last Price':[], 'Strike':[], 'IV':[]}

# Data for calls

for i in calls_Name:
  calls['Contract Name'].append(i)
for i in calls_Exp:
  calls['Expiration'].append(i)
for i in calls_LP:
  calls['Last Price'].append(i)
for i in calls_Strike:
  calls['Strike'].append(i)
for i in calls_IV:
  calls['IV'].append(i)

# Data for puts

for i in puts_Name:
  puts['Contract Name'].append(i)
for i in puts_Exp:
  puts['Expiration'].append(i)
for i in puts_LP:
  puts['Last Price'].append(i)
for i in puts_Strike:
  puts['Strike'].append(i)
for i in puts_IV:
  puts['IV'].append(i)

# Form dataframes

df_calls = pd.DataFrame(data = calls)
df_puts = pd.DataFrame(data = puts)

In [ ]:
##############################
##### Sentiment Analysis #####
##############################

# Fetch general web data for processing

url = 'https://finviz.com/quote.ashx?t='

url = url + ticker
req = Request(url=url,headers={"User-Agent": "Chrome"}) 
response = urlopen(req)    
html = BeautifulSoup(response,"html.parser")
news_table = html.find(id='news-table')
news_table_tr = news_table.findAll('tr')

# Extract date, time, and headlines

for x, table_row in enumerate(news_table_tr):
    a_text = table_row.a.text
    td_text = table_row.td.text
    if x == 10:
        break

# Parse date, time, and headlines into list

news_list = []

for i in news_table.findAll('tr'):    
    text = i.a.get_text() 
    date_scrape = i.td.text.split()
    if len(date_scrape) == 1:
        time = date_scrape[0]      
    else:
        date = date_scrape[0]
        time = date_scrape[1]
    news_list.append([ticker, date, time, text])

# Begin ML sentiment analysis

vader = SentimentIntensityAnalyzer()

# Form dataframes for analysis

columns = ['ticker', 'date', 'time', 'headline']
news_df = pd.DataFrame(news_list, columns=columns)
scores = news_df['headline'].apply(vader.polarity_scores).tolist()
scores_df = pd.DataFrame(scores)
news_df = news_df.join(scores_df, rsuffix='_right')
news_df['date'] = pd.to_datetime(news_df.date).dt.date

mean_scores = news_df.groupby(['ticker','date']).mean()

mean_scores

neg       neu       pos  compound
ticker date                                              
aapl   2022-06-14  0.049591  0.856000  0.094409  0.088741
       2022-06-15  0.095576  0.839879  0.064515 -0.053052
       2022-06-16  0.106563  0.793687  0.099750 -0.023681
       2022-06-17  0.077810  0.860000  0.062190 -0.048262
       2022-06-18  0.029875  0.911375  0.058750  0.050025

In [ ]:
############################
##### Neptune Analysis #####
############################

############################
##### Helper Functions #####
############################

# Root Mean Squared Error
def calculate_rmse(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true-y_pred)**2))                   
    return rmse

# Mean Absolute Percentage Error
def calculate_mape(y_true, y_pred): 
    y_pred, y_true = np.array(y_pred), np.array(y_true)    
    mape = np.mean(np.abs((y_true-y_pred) / y_true))*100    
    return mape

np.random.seed(42)
tf.random.set_seed(42)

api_key = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIyNzk4OGJiNy0yZjQzLTRhMmMtYTk1My1jMzlmN2IxZDgzM2QifQ'

project = neptune.init(api_token=os.getenv(api_key), project_qualified_name='ethanbores/Stock-Analyzer')

url_string = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=%s&outputsize=full&apikey=%s"%(ticker,api_key)

In [ ]:
###############################
##### Decision Calculator #####
###############################

# Determine whether trader should purchase call or put options based on 
# previously acquired and processed data

# Use following variables:
#
# 1) analyst_Rating
# 2) mean_scores

# Analyst ratings are bound such that 1 <= x <= 5 
# where 1 is a 'strong buy' and 5 is a 'sell'

# Compound scores will be used from the mean_scores variable which are bound 
# such that -1 <= x <= 1 where the closer the compound score is to +1, 
# the higher positivity of sentiment

# Identifier for CALL (0) / PUT (1) / STRADDLE (2)
opt = 0

avg_comp = sum(mean_scores['compound']) / len(mean_scores['compound'])

ovr_comp = avg_comp * 50
ovr_analyst = (AR_num * -10) + 60

ovr_rate = ovr_comp + ovr_analyst

print("Average Compound Score:", round(avg_comp, 4))
print("Overall Compound Score:", round(ovr_comp, 4), '\n')

print("Original Analyst Score:", round(AR_num, 4))
print("Adjusted Analyst Score:", round(ovr_analyst, 4), '\n')

if ovr_rate >= 60:
  opt = 0
  print("CALL advised for "+ticker.upper()+" / "+sym_name+" with decision score:", round(ovr_rate, 1), "/ 100")

elif 40 <= ovr_rate < 60:
  opt = 2
  if vix < 40:
    print("SHORT STRADDLE advised for "+ticker.upper()+" / "+sym_name+" with decision score:", round(ovr_rate, 1), "/ 100")
  else:
    print("LONG STRADDLE advised for "+ticker.upper()+" / "+sym_name+" with decision score:", round(ovr_rate, 1), "/ 100")

else:
  opt = 1
  print("PUT advised for "+ticker.upper()+" / "+sym_name+" with decision score:", round(ovr_rate, 1), "/ 100")

Average Compound Score: 0.0028
Overall Compound Score: 0.1377 

Original Analyst Score: 1.9
Adjusted Analyst Score: 41.0 

SHORT STRADDLE advised for AAPL / Apple Inc. with decision score: 41.1 / 100


The Black-Scholes-Merton formula is an estimate of the prices of European call and put options, with the core difference between American and European options being that European options can only be exercised on their one exercise date versus American call options that can be exercised any time up to that expiration date. It's also used only to determine prices of non-dividend paying assets.

The Black–Scholes-Merton formula of value for a European call option is (note: the formula for a European put option is similar)

\begin{aligned}C(S_0,t)&=S_0N(d_1) - Ke^{-r(T-t)}N(d_2)\end{aligned}
where
\begin{aligned} d_1 &= \frac{\ln{\frac{S_0}{K}} + \big(r + \frac{\sigma^2}{2}\big)(T-t)}{\sigma\sqrt{T-t}} \\ d_2 &= d_1 - \sigma\sqrt{(T-t)} \\&= \frac{\ln{\frac{S_0}{K}} + \big(r - \frac{\sigma^2}{2}\big)(T-t)}{\sigma\sqrt{T-t}}\end{aligned}

In [ ]:
################################################
##### Black-Scholes-Merton Equation Solver #####
################################################

# Use following variables:
#
# 1) S_0   = live_price
# 2) K     = calls_Strike / puts_Strike
# 3) t     = time to expiration
# 4) T     = 365
# 5) r     = 0.02
# 6) theta = calls_IV / puts_IV
  
def d1(s0, K, r, theta, t):
  nominator   = np.log(s0/K) + (r + (p2f(theta) ** 2)/2) * (t / 365)
  denominator = theta * (math.sqrt(t / 365))
  return nominator / denominator

def d2(d1, theta, t):
  return d1 - theta * (math.sqrt(t / 365))

def N1(d1):
  return st.norm.ppf(d1)

def N2(d2):
  return st.norm.ppf(d2)

def BSMsolve(S_0, K, r, theta, t):
  return S_0 * N1(d1(S_0, K, r, theta, t)) - K * (math.e ** (-r * (t / 365))) * N2(d2(d1(S_0, K, r, theta, t), theta, t))

In [ ]:
###########################
##### Option Analyzer #####
###########################

# Use following variables:
#
# 1) S_0      = live_price
# 2) K        = calls_Strike / puts_Strike
# 3) t        = calls_Exp / puts_Exp
# 4) T        = 365
# 5) r        = 0.02
# 6) theta    = calls_IV / puts_IV
# 7) Put/Call = opt

res = 0

if opt is 0:
  for i in calls:
    #res = BSMsolve(live_price, calls['Strike'], 0.02, calls['IV'], calls['Expiration'])
    print(res)

elif opt is 1:
  for i in puts:
    #res = BSMsolve(live_price, puts['Strike'], 0.02, puts['IV'], puts['Expiration'])
    print(res)

else:
  for i in calls:
    #res = BSMsolve(live_price, calls['Strike'], 0.02, calls['IV'], calls['Expiration'])
    print(res)

0
0
0
0
0
